In [1]:
import pandas as pd
import numpy as np
import matplotlib as mp
import math as math
import matplotlib.pyplot as plt

# Question 1:

In [2]:
# define function that is u(x,y)
def u(x,y):
    return np.exp(x**6 - y**4)

# define constant for number of samples
N = 10**6

In [3]:
def monte_uniform():
    # get random x and y values based off uniform distribution, and number of samples specified
    x = np.random.uniform(-1, 1, N)
    y = np.random.uniform(-1, 1, N)
    
    # get values based off of randomly sampled x and y
    random_u_values = u(x, y)
    
    # estimate integral area by taking the average of the values, and multiply by area
    integral_est = 4 * np.mean(random_u_values)
    
    # return integral estimate and then return the error 
    return integral_est, integral_est - 4.028423

In [4]:
# create a function for the magnitude of the gradient function
def grad_u_mag(x, y):
    # calc grad of x, y, then the magnitude
    grad_x = 6 * x**5
    grad_y = -4 * y**3
    return np.exp(x**6 - y**4) * np.sqrt(grad_x**2 + grad_y**2)



In [5]:
# apply weights to uniformly sampled values to simuluate proportion density sampling
def monte_weighted():

    # get random x and y values based off uniform distribution, and number of samples specified
    x = np.random.uniform(-1, 1, N)
    y = np.random.uniform(-1, 1, N)

    # get values based off of randomly sampled x and y
    random_u_values = u(x, y)

    # get the weights of the randomly sampled x and y based off the gradient, to get simulate proportional density 
    weights = grad_u_mag(x, y)

    # normalize weights to sum to 0
    normalized_weights = weights / np.sum(weights)

    # esitmate integral
    integral_est = np.sum(random_u_values * normalized_weights) * 4

    # return integral estimate and then return the error
    return integral_est, integral_est - 4.028423


In [6]:
uniform_int_val, uniform_int_error = monte_uniform()
weighted_int_val, weighted_int_error = monte_weighted()

print(f"uniform method gives an integral value of: {uniform_int_val}, with an error of {uniform_int_error}")
print(f"weighted method gives an integral value of: {weighted_int_val}, with an error of {weighted_int_error}")

uniform method gives an integral value of: 4.028910511843306, with an error of 0.00048751184330608766
weighted method gives an integral value of: 5.659618605833079, with an error of 1.6311956058330788


uniform method gives an integral value of: 4.028910511843306, with an error of 0.00048751184330608766

weighted method gives an integral value of: 5.659618605833079, with an error of 1.6311956058330788

# Question 2:

In [7]:
# define N for this problem
N = 2**10

In [8]:
def generate_matrices():
    A = np.random.uniform(-1, 1, (N, N))
    B = np.random.uniform(-1, 1, (N, N))
    return A, B

In [9]:
def add(A, B): return A + B
def sub(A, B): return A - B

def strassen_2level(A, B, level):
    # get size of A being recursively passed down
    n = A.shape[0]
    
    # base case for strassen recursion
    if level == 0 or n == 1:
        return A @ B

    # get mid of the matricies to split it properly
    mid = n // 2
    A11, A12 = A[:mid, :mid], A[:mid, mid:]
    A21, A22 = A[mid:, :mid], A[mid:, mid:]
    B11, B12 = B[:mid, :mid], B[:mid, mid:]
    B21, B22 = B[mid:, :mid], B[mid:, mid:]

    # apply the strassen M transformations
    M1 = strassen_2level(add(A11, A22), add(B11, B22), level-1)
    M2 = strassen_2level(add(A21, A22), B11, level-1)
    M3 = strassen_2level(A11, sub(B12, B22), level-1)
    M4 = strassen_2level(A22, sub(B21, B11), level-1)
    M5 = strassen_2level(add(A11, A12), B22, level-1)
    M6 = strassen_2level(sub(A21, A11), add(B11, B12), level-1)
    M7 = strassen_2level(sub(A12, A22), add(B21, B22), level-1)

    # recombine matriceis according to strassen
    C11 = M1 + M4 - M5 + M7
    C12 = M3 + M5
    C21 = M2 + M4
    C22 = M1 - M2 + M3 + M6

    # assemble final matrix using horizontal and vertical stack and return
    return np.vstack((np.hstack((C11, C12)), np.hstack((C21, C22))))


In [10]:
def inverse_matrix(C):
    # get size of C
    n = C.shape[0]
    # copy matrix so C is not effected
    A = np.copy(C)
    # create identiy matrix of size n
    I = np.identity(n)
    
    for i in range(n):
        # first go through and make sure the pivot is non 0
        if A[i, i] == 0:
            # if 0, go through ever row after and check if the row beneath is has a non 0 pivot in the correct spot
            for j in range(i+1, n):
                # if non 0 new pivot, swap rows in both A and I, as according go gaussian jordan method
                if A[j, i] != 0:
                    I[[i, j]] = I[[j, i]]
                    A[[i, j]] = A[[j, i]]
    
                    # break second loop and continue loop
                    break
        
        # normalize pivot
        pivot = A[i, i]
        # divide whole row by pivot
        A[i] = A[i] / pivot
        I[i] = I[i] / pivot
        
        # apply elimination to other rows by a factor of what what A[j, i]. so if pivot is now 1 and index holds 3 in row underneath,
        # then factor is 3, and the pivot row is multiplied by factor (3) and subtracts the non-pivot row we are transforming
        # reflect change in identity matrix
        for j in range(n):
            if j != i:
                factor = A[j, i]
                I[j] = I[j] - factor * I[i]
                A[j] = A[j] - factor * A[i]
                
    return I

In [11]:
A, B = generate_matrices()

C = strassen_2level(A, B, 2)

C_inv = inverse_matrix(C)


# IF EXPORT NEEDED UNCOMMENT CELL BELOW AND RE-RUN

In [12]:

# np.savetxt("A.csv", A, delimiter=",", fmt="%.3f")
# np.savetxt("B.csv", B, delimiter=",", fmt="%.3f")
# np.savetxt("C.csv", C, delimiter=",", fmt="%.3f")
# np.savetxt("C_inv.csv", C_inv, delimiter=",", fmt="%.3f")


# Question 4:

In [13]:
# For bvp, 
# will guess value for y'(0)
# use rk4 to integrate from x=0 to x=2
# check result at y(2), if not == 0, then adjust guess using secant method
# converge to correct slope


# parameters for problem
h = 2/200
# general list of x values given h step
x_vals = np.arange(0, 2 + h, h)

# convert to first order system
def f(x, y1, y2):
    dy1dx = y2
    dy2dx = -x * y1
    return dy1dx, dy2dx

# using runge kutta method to integrate
def rk4(y1_0, y2_0, x_vals):
    y1 = np.zeros_like(x_vals)
    y2 = np.zeros_like(x_vals)
    y1[0] = y1_0
    y2[0] = y2_0

    for i in range(1, len(x_vals)):
        x = x_vals[i - 1]

        k1_y1, k1_y2 = f(x, y1[i - 1], y2[i - 1])
        k2_y1, k2_y2 = f(x + h / 2, y1[i - 1] + h * k1_y1 / 2, y2[i - 1] + h * k1_y2 / 2)
        k3_y1, k3_y2 = f(x + h / 2, y1[i - 1] + h * k2_y1 / 2, y2[i - 1] + h * k2_y2 / 2)
        k4_y1, k4_y2 = f(x + h, y1[i - 1] + h * k3_y1, y2[i - 1] + h * k3_y2)

        y1[i] = y1[i - 1] + (h / 6) * (k1_y1 + 2 * k2_y1 + 2 * k3_y1 + k4_y1)
        y2[i] = y2[i - 1] + (h / 6) * (k1_y2 + 2 * k2_y2 + 2 * k3_y2 + k4_y2)

    return y1

# shooting method using secant method for root-finding
def shooting_method(s0, s1):
    # init value
    y1_0 = 1
    # max iterations of 100
    for _ in range(100):
        y_s0 = rk4(y1_0, s0, x_vals)[-1]
        y_s1 = rk4(y1_0, s1, x_vals)[-1]

        # update secant
        s_new = s1 - (y_s1 - 2) * (s1 - s0) / (y_s1 - y_s0)

        # check if difference is within acceptable differance, if so then secant has converged and return
        if abs(y_s1 - 2) < 1e-6:
            return rk4(y1_0, s1, x_vals), s1

        # if no convergence replace values and continue
        s0, s1 = s1, s_new

solution, final_slope = shooting_method(0.0, 2.0)

# create df of solutions
table = pd.DataFrame({
    "x": x_vals,
    "y": np.round(solution, 4)
})

table


,x,y
0,0.00,1.0000
1,0.01,1.0224
2,0.02,1.0448
3,0.03,1.0672
4,0.04,1.0896
...,...,...
196,1.96,2.1198
197,1.97,2.0905
198,1.98,2.0607
199,1.99,2.0306


In [15]:
table.to_csv('problem_4_results.csv')